<a href="https://colab.research.google.com/github/GArdennes/Research-Studies/blob/main/Microservice_Placement_Optimization_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 76.8 MB/s eta 0:00:00


In [ ]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 122531 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-2_amd64.deb ...
Unpacking glpk-utils (4.65-2) ...
Setting up libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Setting up libamd2:amd64 (1:5.7.1+

In [ ]:
#import modules
import pulp

In [ ]:
# Define decision variables
num_microservices = 4
num_edge_nodes = 4
x = pulp.LpVariable.dicts("x", [(i,j) for i in range(num_microservices) for j in range(num_edge_nodes)], cat=pulp.LpBinary)
t = pulp.LpVariable.dicts("t", [(i,j) for i in range(num_microservices) for j in range(num_edge_nodes)], cat=pulp.LpContinuous)

microservices = [
    {"name": "microservice1", "category": "1", "ram": 512, "input": 256, "output": 256, "cpu": 50, "dependencies": []},
    {"name": "microservice2", "category": "2", "ram": 1024, "input": 512, "output": 512, "cpu": 100, "dependencies": ["microservice1"]},
    {"name": "microservice3", "category": "1","ram": 2048, "input": 1024, "output": 1024, "cpu": 200, "dependencies": []},
    {"name": "microservice4", "category": "2","ram": 4096, "input": 2048, "output": 2048, "cpu": 400, "dependencies": []}
]


edge_nodes = [
    {"name": "edge_node1", "speed": 100, "ram": 2048, "uplink": 512, "downlink": 512, "busy_power": 100, "idle_power": 50, "cpu": 1000},
    {"name": "edge_node2", "speed": 200, "ram": 4096, "uplink": 1024, "downlink": 1024, "busy_power": 150, "idle_power": 75, "cpu": 2000},
    {"name": "edge_node3", "speed": 300, "ram": 8192, "uplink": 2048, "downlink": 2048, "busy_power": 200, "idle_power": 100, "cpu": 2500},
    {"name": "edge_node4", "speed": 400, "ram": 16384, "uplink": 4096, "downlink": 4096, "busy_power": 250, "idle_power": 100, "cpu": 4000}
]

dependencies = [
    [0, 1, 1, 0],  # microservice 1 depends on 2 and 3
    [0, 0, 0, 1],  # microservice 2 depends on 4
    [0, 0, 0, 0],  # microservice 3 has no dependencies
    [0, 0, 0, 0],  # microservice 4 has no dependencies
]


In [ ]:
# Set up problem
problem = pulp.LpProblem("Microservice_Placement", pulp.LpMinimize)

In [ ]:
#Define function
def is_edge_node_available(microservice, edge_node):
    if microservice["ram"] > edge_node["ram"]:
        return False
    if microservice["cpu"] > edge_node["cpu"]:
        return False
    if microservice["input"] > edge_node["downlink"]:
        return False
    if microservice["output"] > edge_node["uplink"]:
        return False
    return True


In [ ]:
# Define objective function
weights = {"latency": 0.5, "throughput": -0.5}  # hyperparameters
latency = [[10, 20, 30, 40], [15, 25, 35, 45], [20, 30, 40, 50], [25, 35, 45, 55]]
throughput = [[100, 200, 300, 400], [150, 250, 350, 450], [200, 300, 400, 500], [250, 350, 450, 550]]

# Define objective function
obj = pulp.lpSum([weights["latency"] * latency[i][j] * x[(i,j)] + weights["throughput"] * throughput[i][j] * x[(i,j)] for i in range(num_microservices) for j in range(num_edge_nodes)])
problem += obj

In [ ]:
# Constraint 1: Each microservice is placed on exactly one edge node
for i in range(num_microservices):
    problem += pulp.lpSum([x[(i,j)] for j in range(num_edge_nodes)]) == 1

In [ ]:
# Constraint 2: An edge node cannot exceed its resource capacity
for j in range(num_edge_nodes):
    problem += pulp.lpSum([microservices[i]["ram"] * x[(i,j)] for i in range(num_microservices)]) <= edge_nodes[j]["ram"]

In [ ]:
# Constraint 3: A microservice cannot be placed on an unavailable edge node
for i in range(num_microservices):
    for j in range(num_edge_nodes):
        if not is_edge_node_available(microservices[i], edge_nodes[j]):
            problem += x[(i,j)] == 0

In [ ]:
#Constraint 4: Dependencies between microservices
for k in range(num_microservices):
            if k != i:
                # Microservices i and k cannot be placed on the same edge node if i depends on k
                if k in microservices[i]["dependencies"]:
                    problem += x[(i,j)] + x[(k,j)] <= 1

In [ ]:
#Constraint 5: Recovery after timeout constraint
timeout = [0.01, 0.02, 0.03, 0.04]
for i in range(num_microservices):
    for j in range(num_edge_nodes):
        problem += t[(i,j)] >= x[(i,j)] * timeout[i], f"Timeout{i}{j}"

In [ ]:
status = problem.solve(pulp.GLPK_CMD())

In [ ]:
# Print the status of the solution
print("Status:", pulp.LpStatus[problem.status])

Status: Optimal


In [ ]:
# Print the value of the objective function
print("Objective value:", pulp.value(problem.objective))

Objective value: -855.0


In [ ]:
# Print the decision variables
for i in range(num_microservices):
    for j in range(num_edge_nodes):
        print(f"x[{i},{j}] = {x[(i,j)].value()}, t[{i},{j}] = {t[(i,j)].value()}")

x[0,0] = 0, t[0,0] = 0.0
x[0,1] = 0, t[0,1] = 0.0
x[0,2] = 0, t[0,2] = 0.0
x[0,3] = 1, t[0,3] = 0.01
x[1,0] = 0, t[1,0] = 0.0
x[1,1] = 0, t[1,1] = 0.0
x[1,2] = 0, t[1,2] = 0.0
x[1,3] = 1, t[1,3] = 0.02
x[2,0] = 0, t[2,0] = 0.0
x[2,1] = 0, t[2,1] = 0.0
x[2,2] = 0, t[2,2] = 0.0
x[2,3] = 1, t[2,3] = 0.03
x[3,0] = 0, t[3,0] = 0.0
x[3,1] = 0, t[3,1] = 0.0
x[3,2] = 0, t[3,2] = 0.0
x[3,3] = 1, t[3,3] = 0.04
